# Notebook 14: Uncertainties

Uncertainties are everywhere in geosciences. Everytime we take a measurement, there are uncertainties associated to this measurement, and everytime we look at data (e.g. seismic data), there are uncertainties associated to how these data were acquired, processed, displayed and interpreted.

Uncertainties (call them errors if you want) propagate through any calculation where we use measurements (or observations) with errors. If these measurements are statistically independent (they are uncorrelated with the magnitude and error of all other measurements), the general formula to propagate the error is:


$$\sigma_z=\sqrt{\left(\frac{\partial z}{\partial a}\right)^2\left(\sigma_a\right)^2+\left(\frac{\partial z}{\partial b}\right)^2\left(\sigma_b\right)^2+\left(\frac{\partial z}{\partial c}\right)^2\left(\sigma_c\right)^2+\cdots}$$


where $z$ is a multi-variable function $z=f(a,b,c,...)$ that depends on the measurements $a$, $b$, $c$, etc. $\sigma_z$ is the uncertainty of $z$, and $\sigma_a$, $\sigma_b$, $\sigma_c$, etc., are the uncertainty of the measurements.

It is easy to calculate the formula above for simple cases (e.g. sum or multiplication of two variables), but it is difficult when the formulas are more complicated, or when we need to use several formulas to get the result.

Fortunately in Python, there is a library to deal with uncertainties and propagate errors as in the equation above. This library is called [uncertainties](https://pythonhosted.org/uncertainties/). So if you have not installed `uncertainties`, please do so by running the cell below:

In [ ]:
# run this cell if uncertainties is not installed
import sys
!{sys.executable} -m pip install --upgrade uncertainties

In this notebook, I illustrate the use of the `uncertainties` library using three examples.

## Example 1: Bed thickness

The first example is problem 4 in Chapter 2 of [Ragan (2009)](https://www.cambridge.org/core/books/structural-geology/4D631885C9FBBCDEF90C555445ED1160#):

The orientation of a sandstone unit is 245/35 (right hand rule convention). A horizontal traverse with a bearing of N10E made from the bottom to the top of the unit measured 125 m.

- Calculate the thickness of the unit.

- If the uncertainty in dip is 2°, the uncertainty of the traverse direction is 1°, and the uncertainty in the measured length is 0.5%, what is the uncertainty in the calculated thickness?

The figure below shows on map view the variables (measurements) for this problem, and the equation we can use to determine the thickness of the sandstone:

<img src="../figures/ss_thickness.png" alt="ss_thickness" width="600" style="display: block; margin: 0 auto"/><br><br>

Let's solve first this problem the hard way, by computing the partial derivatives and solving for the error in quadrature formula:

In [1]:
import math # import math

rad = 180.0/math.pi # a radian in degrees

# Problem 4, chapter 2 of Ragan
l = 125 # transect length
l_u = l * 0.005 # uncertainty in transect length
dip = 35 / rad # dip in radians
dip_u = 2 / rad # uncertainty in dip in radians
beta = 55 / rad # angle of traverse with strike line in radians
beta_u = 1 / rad # uncertainty in beta in radians

# compute thickness of bed, Eq. 2.2 of Ragan
t = l*math.sin(beta)*math.sin(dip)
print(f"Thickness = {t:.1f} m")

Thickness = 58.7 m


In [2]:
# Compute error in thickness

# partial derivatives, here we need to use calculus
ptl = math.sin(beta)*math.sin(dip) # partial derivative of t with respect to l
ptb = l*math.cos(beta)*math.sin(dip) # partial derivative of t with respect to beta
ptd = l*math.sin(beta)*math.cos(dip) # partial derivative of t with respect to dip

# error in quadrature formula
t_u = math.sqrt((ptl*l_u)**2 + (ptb*beta_u)**2 + (ptd*dip_u)**2)

# Output result
print(f"Thickness = {t:.1f} +/- {t_u:.1f} m")

Thickness = 58.7 +/- 3.0 m


So the error in thickness is about 5% the computed thickness. Now let's solve this problem using the `uncertainties` library. For that, we will need to create `ufloat`s (floats with uncertainties), and use `umath` (math with uncertainty):

In [3]:
# import ufloat and umath functions
from uncertainties import ufloat # float with uncertainties
from uncertainties import umath # math with uncertainties

# define parameters with uncertainties
l = ufloat(125, 125*0.005)
dip = ufloat(35, 2) / rad
beta = ufloat(55, 1) / rad

# compute thickness
t = l*umath.sin(beta)*umath.sin(dip)

# output result
print(f"Thickness = {t:.1f} m")

Thickness = 58.7+/-3.0 m


We got the same result than above but in a much more efficient way. We also demonstrated that the `uncertainties` package works. 

## Python functions

The functions in:

- [angles_u](../functions/angles_u.py) 

- [three_points_u](../functions/three_points_u.py)

- [true_thickness_u](../functions/true_thickness_u.py)

are similar to the functions in [angles](../functions/angles.py), [three_points](../functions/three_points.py), and [true_thickness](../functions/true_thickness.py), but they incorporate uncertainties.

## Example 2

Let's solve the first problem in [notebook 3](nb3_vector_operations.ipynb):

Two limbs of a chevron fold (A and B) have orientations (RHR) as follows:

Limb A = 120/40 \
Limb B = 250/60 

Determine: 

1. the trend and plunge of the hinge line of the fold, 
2. the rake of the hinge line in limb A, 
3. the rake of the hinge line in limb B

This time, however, let's assume that the uncertainty in strike is 4° and in dip is 2°.

In [4]:
# import numpy
import numpy as np

# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath(os.path.join("..", "functions")))

# import our functions
from angles_u import angle_bw_lines_u, int_bw_planes_u

# Strike and dip of the limbs in radians
stk1, dip1 = np.radians([120, 40]) # SW dipping limb
stk2, dip2 = np.radians([250, 60]) # NW dipping limb

# Errors in strike and dip in radians
ustr, udip = np.radians([4, 2])

# Create the input values with uncertainties
stk1 = ufloat(stk1, ustr) # stk1 = stk1 +/-ustr
dip1 = ufloat(dip1, udip) # dip1 = dip1 +/-udip
stk2 = ufloat(stk2, ustr) # stk2 = stk2 +/-ustr
dip2 = ufloat(dip2, udip) # dip2 = dip2 +/-udip

# (1) Chevron folds have planar limbs. The hinge
# of the fold is the intersection of the limbs
htrd, hplg = int_bw_planes_u(stk1,dip1,stk2,dip2)
print(f"Hinge trend = {htrd*rad:.1f}, plunge {hplg*rad:.1f}")

# The rake of the hinge on either limb is the angle
# between the hinge and the strike line on the limb.
# This line is horizontal and has plunge = 0
plg = ufloat(0, udip) # plg = 0 +/-udip

# (2) Rake of hinge on SW dipping limb
ang = angle_bw_lines_u(stk1,plg,htrd,hplg)
print(f"Rake of hinge on SW dipping limb = {ang*rad:.1f} E")

# (3) Rake of hinge on NW dipping limb
ang = angle_bw_lines_u(stk2,plg,htrd,hplg)
print(f"Rake of hinge on NW dipping limb = {ang*rad:.1f} W")

Hinge trend = 265.8+/-3.3, plunge 25.3+/-2.6
Rake of hinge on SW dipping limb = 138.4+/-4.4 E
Rake of hinge on NW dipping limb = 29.5+/-3.8 W


## Example 3

Let's solve the problem in [notebook 4](nb4_three_points.ipynb):

The coordinates of the coal seams at the three boreholes A, B and C are:

<div align="center">

| Borehole | Lower seam | Upper seam |
| -------- | ---------- | ---------- |
| A | [393, 2374, 550] | [393, 2374, 950] |
| B | [1891, 2738, 650] | [1891, 2738, 1050] |
| C | [2191, 1037, 450] | [2191, 1037, 850] |

</div>

Compute the strike and dip of the lower seam. This time, however, let's assume that the uncertainty in east and north coordinates is 10 m, and in elevation is 5 m.

In [5]:
# Import function three_points_u
from three_points_u import three_points_u

# ENU coordinates of the three points
# with uncertainties in E-N = 10, and U = 5
p1 = np.array([ufloat(393, 10), ufloat(2374, 10), 
               ufloat(550, 5)])
p2 = np.array([ufloat(1891, 10), ufloat(2738, 10), 
               ufloat(650, 5)])
p3 = np.array([ufloat(2191, 10), ufloat(1037, 10), 
               ufloat(450, 5)])

stk, dip = three_points_u(p1,p2,p3)
print(f"Lower coal seam strike = {stk*rad:.1f}, dip = {dip*rad:.1f}")

Lower coal seam strike = 106.4+/-1.6, dip = 7.4+/-0.3


## Example 4

Let's solve the problem in [notebook 5](nb5_thickness.ipynb), which involves computing the stratigraphic thickness of conformable sedimentary units on a map (please refer to [notebook 5](nb5_thickness.ipynb)). This time however, the uncertainties in strike and dip are 4° and 2°, respectively, the uncertainty in east and north coordinates is 10 m, and the uncertainty in elevation is 5 m.

Here is the solution:

In [6]:
# import true_thickness_u function
from true_thickness_u import true_thickness_u

 # create the strike and dip with uncertainties, in radians
stk = ufloat(84.5, 4) / rad
dip = ufloat(22.5, 2) / rad

# ENU coordinates of the points
# with uncertainties in E-N = 10, and U = 5
p1 = np.array([ufloat(1147, 10), ufloat(3329, 10), 
               ufloat(400, 5)]) 
p2 = np.array([ufloat(1323, 10), ufloat(2362, 10), 
               ufloat(500, 5)]) 
p3 = np.array([ufloat(1105, 10), ufloat(1850, 10), 
               ufloat(400, 5)]) 
p4 = np.array([ufloat(1768, 10), ufloat(940, 10), 
               ufloat(300, 5)]) 
p5 = np.array([ufloat(1842, 10), ufloat(191, 10), 
               ufloat(200, 5)])

# compute the thickness of the units
thickT = true_thickness_u(stk, dip, p2, p1)
thickS = true_thickness_u(stk, dip, p3, p2)
thickR = true_thickness_u(stk, dip, p4, p3)
thickQ = true_thickness_u(stk, dip, p5, p4) 
print("Thickness of unit T = {:.1f} m".format(thickT))
print("Thickness of unit S = {:.1f} m".format(thickS))
print("Thickness of unit R = {:.1f} m".format(thickR))
print("Thickness of unit Q = {:.1f} m".format(thickQ))

Thickness of unit T = 467.2+/-31.5 m
Thickness of unit S = 94.6+/-20.4 m
Thickness of unit R = 278.6+/-37.0 m
Thickness of unit Q = 195.6+/-27.0 m


For the thinnest unit S, the uncertainty in thickness is about 20% the thickness of the unit! Uncertainties are really important.